### Imports

In [ ]:
# Scrap 
import requests
import ScraperFC as sfs

# Data Managment 
import pandas as pd
import numpy as np

# Viz
from mplsoccer import Pitch, VerticalPitch
import matplotlib.pyplot as plt 

# Others
import os 
from PIL import Image
from io import BytesIO
import tweepy

In [ ]:
# Fonts and colors 
from matplotlib import font_manager
locations = ['/Users/emilio/Documents/3- Resources/lightstats/fonts/Montserrat']
font_files = font_manager.findSystemFonts(fontpaths=locations)
for file in font_files:
    font_manager.fontManager.addfont(file)
font = 'Montserrat'

# Define the colors using hexadecimal representations
background_color = '#F7F7F7' 
title_color = '#000000'   
text_color = '#333333'  
sub_text_color = '#808080'   
accent_color1 = '#4682B4'    
accent_color2 = '#ADD8E6' 

### Scrap

In [ ]:
# Scrap sofascore
sofascore = sfs.Sofascore()

sofascore_url = 'https://www.sofascore.com/real-madrid-fc-bayern-munchen/xdbsEgb#id:12173506'
sofascore_url = sofascore_url.replace('id:', '')

### Data

In [ ]:
# Get opponent team for the SofaScore POM
def get_opponent_team(df, playerName):
    # Filter rows where playerName matches
    player_df = df[df['name'] == playerName]

    # Get the player's team name
    player_team = player_df['team'].unique()[0]

    # Find the opponent team
    opponent_team = df[df['team'] != player_team]['team'].unique()[0]

    return opponent_team

#### Average Positions Data

In [ ]:
# DataFrames for the Average Positions 
local = sofascore.get_players_average_positions(sofascore_url)[0]
localPositions = local[['jerseyNumber', 'averageX', 'averageY']]
localname = local['team'].unique()[0]

away = sofascore.get_players_average_positions(sofascore_url)[1]
awayPositions = away[['jerseyNumber', 'averageX', 'averageY']]
awayname = away['team'].unique()[0]

#### Heatmap data

In [ ]:
# DataFrame of both teams 
teamsDf = pd.concat([local, away])
teamsDf['name'].unique()

In [ ]:
playerName = 'Vinícius Júnior'
opponent_team = get_opponent_team(teamsDf, playerName)
# Heatmap DataFrame (POM)
player_heatmap = sofascore.get_player_heatmap(sofascore_url, playerName)

### Average Positions

In [ ]:
fig, ax = plt.subplots()
pitch = Pitch(pitch_type='opta', half=False)
pitch.draw(ax=ax)

pitch.scatter(localPositions['averageX'], localPositions['averageY'], ax=ax, s=100, label=f'{localname}')

# Home shirt numbers
for i, jersey_number in enumerate(localPositions['jerseyNumber']):
    ax.annotate(jersey_number, (localPositions['averageX'].iloc[i], localPositions['averageY'].iloc[i]), fontsize=10, ha='center', va='center')

pitch.scatter(100-awayPositions['averageX'], 100-awayPositions['averageY'], ax=ax, s=100, label=f'{awayname}')

# Away shirt numbers
for i, jersey_number in enumerate(awayPositions['jerseyNumber']):
    ax.annotate(jersey_number, (100-awayPositions['averageX'].iloc[i], 100-awayPositions['averageY'].iloc[i]), fontsize=10, ha='center', va='center')

ax.set_title(f'{localname} vs {awayname} average positions', va='center')
legend = plt.legend(loc='upper right', fontsize='small')
legend.set_bbox_to_anchor((1.3, 1))  # Ajusta la posición de la leyenda ligeramente fuera del gráfico

plt.show()


### Heatmap

In [ ]:
""" Heatmap visualization of pomSS """
fig, ax = plt.subplots(figsize=(16,9))
fig.set_facecolor(color=background_color)

cmap = 'hot'
pitch = Pitch(pitch_type='opta', pitch_color=background_color, line_color=sub_text_color)
pitch.draw(ax=ax)
pitch.kdeplot(player_heatmap.x, player_heatmap.y, ax=ax,
              levels=100,
              shade=True,
              zorder=-1,
              shade_lowest=True,
              cmap='OrRd'
)
plt.gca()

ax.set_title(f'{playerName} heatmap vs {opponent_team}', fontsize=20, fontfamily=font, color=title_color, weight='bold')

arrow = FancyArrowPatch((40, -2), (60, -2), mutation_scale=20, color=sub_text_color, arrowstyle='-|>')
ax.add_patch(arrow)
ax.text(50, -5, 'Attack', fontsize=12, ha='center', fontfamily=font, color=sub_text_color)